In [1]:
using StatsBase

In [2]:
buf = [[2,3,5], [1], [4,6,7,8,9,3, 2, 1, 1, 3]]

3-element Vector{Vector{Int64}}:
 [2, 3, 5]
 [1]
 [4, 6, 7, 8, 9, 3, 2, 1, 1, 3]

In [5]:
test = sample(buf, weights([length(a) for a in buf]), 8, replace=true)

8-element Vector{Vector{Int64}}:
 [2, 3, 5]
 [2, 3, 5]
 [4, 6, 7, 8, 9, 3, 2, 1, 1, 3]
 [4, 6, 7, 8, 9, 3, 2, 1, 1, 3]
 [4, 6, 7, 8, 9, 3, 2, 1, 1, 3]
 [1]
 [4, 6, 7, 8, 9, 3, 2, 1, 1, 3]
 [4, 6, 7, 8, 9, 3, 2, 1, 1, 3]

In [8]:
using LinearAlgebra
using Flux
using CUDA

In [9]:
CUDA.functional()

true

In [12]:
input = 60
middle = 256
output = 7
depth = 16

16

In [10]:
tanh10(x) = Float32(10)*tanh(x)
tanh2(x) = Float32(2)*tanh(x)

tanh2 (generic function with 1 method)

In [13]:
#model_cpu = Chain(Dense(input, middle), [Chain(Parallel(+, Chain(BatchNorm(middle), Dense(middle, middle, relu)),Dense(middle, middle, relu)), identity) for i in 1:depth], Base.Flatten , Parallel(vcat,Chain(Dense(middle, middle, tanh),Dense(middle, output)), Dense(middle, 1)))
model_cpu = Chain(Dense(input, middle), Tuple(Chain(Parallel(+, Chain(BatchNorm(middle), Dense(middle, middle, relu)),Dense(middle, middle, relu)), identity) for i in 1:depth)..., Flux.flatten , Parallel(vcat,Chain(Dense(middle, middle, relu),Dense(middle, output)), Dense(middle, 1, tanh10)))

Chain(
  Dense(60 => 256),                     # 15_616 parameters
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      Dense(256 => 256, relu),          # 65_792 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      Dense(256 => 256, relu),          # 65_792 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      Dense(256 => 256, relu),          # 65_792 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 =>

In [14]:
model_gpu = gpu(model_cpu)

Chain(
  Dense(60 => 256),                     # 15_616 parameters
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      Dense(256 => 256, relu),          # 65_792 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      Dense(256 => 256, relu),          # 65_792 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      Dense(256 => 256, relu),          # 65_792 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 =>

In [15]:
input_cpu = rand(Float32, input, 100)
input_gpu = gpu(input_cpu)

60×100 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.143417   0.661526   0.300275  …  0.21115    0.390488  0.412317
 0.0827365  0.357492   0.675625     0.706847   0.953116  0.952325
 0.41502    0.711086   0.285852     0.840857   0.211019  0.0876541
 0.213793   0.419338   0.166268     0.724749   0.812207  0.305395
 0.837255   0.315357   0.348232     0.381508   0.580714  0.222859
 0.884044   0.75536    0.151499  …  0.392569   0.569844  0.438391
 0.0397006  0.943632   0.973726     0.792938   0.252825  0.194502
 0.755032   0.0695049  0.917548     0.576288   0.987549  0.134156
 0.731111   0.179743   0.504391     0.713739   0.322203  0.0667186
 0.181318   0.882143   0.160419     0.450602   0.247288  0.999009
 ⋮                               ⋱                       
 0.575558   0.327496   0.791497     0.795949   0.285617  0.498383
 0.540626   0.16058    0.277133     0.474717   0.76029   0.38376
 0.567841   0.955849   0.896216     0.728827   0.993513  0.189484
 0.980667   0.114096   0.59349  

In [17]:
using BenchmarkTools

In [18]:
@benchmark model_cpu(input_cpu)

BenchmarkTools.Trial: 821 samples with 1 evaluation.
 Range (min … max):  5.325 ms … 17.850 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     5.514 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.086 ms ±  1.190 ms  ┊ GC (mean ± σ):  8.66% ± 12.39%

    ▁█                                                        
  ▅▅██▇▄▃▄▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▄▄▅▄▃▂▂▂▂ ▂
  5.33 ms        Histogram: frequency by time         8.2 ms <

 Memory estimate: 9.84 MiB, allocs estimate: 397.

In [19]:
@benchmark model_gpu(input_gpu)

BenchmarkTools.Trial: 3660 samples with 1 evaluation.
 Range (min … max):  1.159 ms … 71.857 ms  ┊ GC (min … max): 0.00% … 22.89%
 Time  (median):     1.225 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.363 ms ±  3.004 ms  ┊ GC (mean ± σ):  2.15% ±  0.96%

                ▃▃▄▆▆▇████▇▆▆▅▄▂▂▁▁        ▁ ▂▁ ▁ ▁  ▁       ▂
  ▅▆▄▅▅▅▄▄▁▅▄▁█▇█████████████████████▇▆██████████████████▇██ █
  1.16 ms      Histogram: log(frequency) by time     1.33 ms <

 Memory estimate: 91.80 KiB, allocs estimate: 2698.

In [24]:
target_cpu = rand(Float32, output+1, 100)
target_gpu = gpu(target_cpu)

8×100 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.442336  0.766151   0.843949   0.54925   …  0.222884   0.499321   0.913901
 0.94984   0.112964   0.751297   0.957962     0.623886   0.528018   0.888046
 0.921804  0.139142   0.750826   0.92957      0.206175   0.472037   0.684832
 0.522072  0.638452   0.406902   0.225323     0.0684221  0.628108   0.722288
 0.5117    0.703904   0.552616   0.189341     0.675611   0.676573   0.704262
 0.799998  0.730623   0.0191956  0.711513  …  0.907321   0.762375   0.663861
 0.991818  0.0983187  0.734487   0.657394     0.301702   0.829439   0.716045
 0.762924  0.788548   0.0635926  0.478378     0.452432   0.0764909  0.614054

In [21]:
training_step=2000

2000

In [25]:
opt=ADAM()

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [31]:
@time for it in 1:training_step
    Flux.train!(Flux.mse, Flux.params(model_cpu), [(model_cpu(input_cpu), target_cpu)], opt)
end

 12.846838 seconds (1.99 M allocations: 19.368 GiB, 11.53% gc time)


In [28]:
opt=ADAM()

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [30]:
@time for it in 1:training_step
    Flux.train!(Flux.mse, Flux.params(model_gpu), [(model_gpu(input_gpu), target_gpu)], opt)
end

  3.347413 seconds (7.34 M allocations: 323.953 MiB, 2.62% gc time)
